In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx to Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx
User uploaded file "Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx" with length 6481955 bytes


In [2]:
import pandas as pd
import numpy as np

#Read dataset
features =  pd.read_excel("Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx")

Rimpiazziamo i codici relativi a determinate feature con i nomi delle feature, così da poter utilizzare la funzione get_dummies() ed ottenere un one-hot-encoding

In [3]:
features["Diploma_scuola_superiore"] = features["Diploma_scuola_superiore"].map({1: "Classico", 2: "Scientifico", 3: "Linguistico", 4: "Magistrale", 5: "Artistico", 6: "Tecnico", 7: "Professionale", 8: "Altro_italiano", 9: "Estero", 99: "Non_disponibile"})
features["area_geografica_scuolasuperiore"] = features["area_geografica_scuolasuperiore"].map({1: "Emilia_romagna", 2: "Nord", 3: "Centro", 4: "Sud_isole", 5: "Estero", 99: "Non_disponibile"})
features["area_geografica_residenza"] = features["area_geografica_residenza"].map({1: "Emilia_romagna", 2: "Nord", 3: "Centro", 4: "Sud_isole", 5: "Estero", 99: "Non_disponibile"})
features["Ambito"] = features["Ambito"].map({1: "Economia", 2: "Farmacia", 3: "Giurisprudenza", 4: "Ingegneria", 5: "Lingue", 6: "Medicina", 7: "Veterinaria", 8: "Psicologia", 9: "Scienze", 10: "Scienze_agroalimanetari", 11: "Scienze_educazione_formazione", 12: "Scienze_motorie", 13: "Scienze_politiche", 14: "Scienze_statistiche", 15: "Sociologia", 16: "Studi_umanistici"})
features = pd.get_dummies(features)

Create and add age in DataFrame

In [4]:
from datetime import datetime, date

def getAge(dataNascita, coorte):
    ts = pd.to_datetime(str(dataNascita)) 
    born = ts.strftime('%d/%m/%Y')
    born = datetime.strptime(born, "%d/%m/%Y").date()
    date = "31/12/"+str(coorte)
    dataAttuale = datetime.strptime(date, "%d/%m/%Y").date()
    age = dataAttuale.year - born.year - ((dataAttuale.month, dataAttuale.day) < (born.month, born.day))
    return age

#add new column with NaN
features["Eta"] = np.nan

for ind in features.index:
    studente = features.iloc[ind]
    dataNascita = studente["DataNascita"]
    coorte = studente["Coorte"]
    #fill the new column with age 
    features.at[ind,'Eta'] = getAge(dataNascita, coorte)


Abbiamo sostituito i valori NaN della feature "voto_scuola_superiore" con la media. Inoltre sono state droppate le colonne non utili ai fini dell'addestramento, ed infine eliminate le tuple contente almeno un NaN

In [5]:
mean_value = features['voto_scuola_superiore'].mean()
features['voto_scuola_superiore'] = features['voto_scuola_superiore'].fillna(mean_value)
features = features.drop('ID_Studente', axis = 1)
features = features.drop('Classe_ISEE', axis = 1)
features = features.drop('DataNascita', axis = 1)
features = features.drop('OFA_superati', axis = 1)
features = features.drop('Coorte', axis = 1)
features = features.dropna()

In [6]:
from sklearn.model_selection import train_test_split as tts
#Dividing the data into attributes and labels
X = features.drop('Abbandoni', axis=1) #sostituire final con features
y = features['Abbandoni'] #sostituire final con features
#Dividing data into training and test sets
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.20, random_state=42)

In [12]:
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error,r2_score
from sklearn import metrics
model = RandomForestClassifier(n_estimators = 207, random_state = 42)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=207, random_state=42)

In [13]:
predictions = model.predict(X_test)
print(predictions)

print(r2_score(y_test,predictions))
print("Evaluating the Algorithm:\n")
print(metrics.confusion_matrix(y_test,predictions))
print(metrics.classification_report(y_test,predictions))


print("Evaluating Prediction Accuracy:\n")
print("Accuracy:", metrics.accuracy_score(y_test, predictions))

#print(*predictions, sep ="\n")

[0 0 0 ... 0 0 0]
-0.13047872569073493
Evaluating the Algorithm:

[[4398   28]
 [ 416   15]]
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      4426
           1       0.35      0.03      0.06       431

    accuracy                           0.91      4857
   macro avg       0.63      0.51      0.51      4857
weighted avg       0.86      0.91      0.87      4857

Evaluating Prediction Accuracy:

Accuracy: 0.9085855466337245
